## Esercizio Lab Hanks

### Librerie

In [ ]:
import spacy
from nltk.wsd import lesk
from collections import defaultdict
from rich.console import Console
from rich.table import Table

### Elaborazione del testo

Nella funzione "elaborate_sentences" estraiamo le frasi dal file sentences.txt sostituendo al posto dei nomi propri la parola "person".

In [ ]:
def elaborate_sentences(verb):
    sentences = set()
    with open('esercizio3/sentences.txt', 'r', encoding="utf8") as values:
        
        for sentence in values:
            if any(item in sentence for item in verb):
                sentences.add(sentence)
         
    personal_name = set()
    with open('esercizio3/nomi_persone.txt', 'r', encoding="utf8") as x:
        personal_name = x.read().split('\n')
    
    list_sentences = list(sentences)
    for count,sentence in enumerate(list_sentences):  
        for word in sentence.split() :
            if word in personal_name:
                    list_sentences[count] = sentence.replace(word, 'person')
                    
    return list_sentences

### Cluster

Per l'implementazione della teoria di Hanks è stato preso in considerazione il verbo "take". Le sentenze sono state lemmatizzate e filtrati solo i nomi e pronomi, successivamente è stato applicato l'algortimo di lesk per disambiguare le parole per poi estrarre super-sense.

In [ ]:
def cluster(sentences,verb):
    left_list=[]
    right_list=[]
    
    sp = spacy.load('en_core_web_md')
    for sentence in sentences:
        tokens = sp(sentence) #parsifico e tokenizzo
        filter_list=[]

        for token in tokens:
            if any(item in token.lemma_ for item in verb):
               
                for child in token.children:
                    match child.pos_:
                        case "NOUN":
                           filter_list.append(child.text)
                        case "PRON":
                            if child.text == 'it' or child.text == 'It':
                                filter_list.append('thing')                                 
                            elif child.text == 'they' or child.text == 'They':
                                continue
                            else:
                                filter_list.append('person')

                if len(filter_list) == 2: #procedo solo se ho 2 argomenti (soggetto, oggetto) sem.type
                    x = lesk(sentence,filter_list[0])
                    y = lesk(sentence,filter_list[1])

                    if x and y is not None: #prendo il super-sence
                        left_list.append(x.lexname()) 
                        right_list.append(y.lexname())
    
    left_to_right = defaultdict(set)
    right_to_left = defaultdict(set)
    for count, item in enumerate(left_list): #faccio gli accoppiamenti tra tra i sem.type
        left_to_right[item].add(right_list[count])
    for count, item in enumerate(right_list):
        right_to_left[item].add(left_list[count])

    print_tables(left_list,right_list,left_to_right,right_to_left)

### Risultati

In [4]:
def print_tables(left_list,right_list,left_to_right,right_to_left):
    console = Console()
    table_l = Table(show_header=True, title="TAKE left to right", show_lines=True)
    table_l.add_column("Frequenza")
    table_l.add_column("Sinistra")
    table_l.add_column("Destra")

    for item in left_to_right:  #conto la frequenza delle coppie e metto in riga nella tabella
        fre = left_list.count(item)
        table_l.add_row(str(fre)+'/'+str(len(left_list)),item, str(left_to_right[item]))
    console.print(table_l)

    table_r = Table(show_header=True, title="TAKE right to left", show_lines=True)
    table_r.add_column("Frequenza")
    table_r.add_column("Sinistra")
    table_r.add_column("Destra")
    for item in right_to_left:
        fre = right_list.count(item)
        table_r.add_row(str(fre)+'/'+str(len(right_list)),item, str(right_to_left[item]))
    console.print(table_r)

                                                TAKE left to right                                                 
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Frequenza ┃ Sinistra           ┃ Destra                                                                         ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 9/120     │ noun.person        │ {'noun.possession', 'verb.body', 'noun.communication', 'noun.artifact',        │
│           │                    │ 'noun.act'}                                                                    │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 6/120     │ noun.group         │ {'noun.person', 'noun.location', 'noun.communication', 'noun.event',           │
│           │                    │ 'noun.attribute'}                                                              │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 11/120    │ noun.act           │ {'noun.location', 'noun.time', 'noun.act'}                                     │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 54/120    │ noun.communication │ {'verb.change', 'verb.body', 'noun.event', 'verb.consumption',                 │
│           │                    │ 'noun.artifact', 'noun.cognition', 'verb.motion', 'noun.location',             │
│           │                    │ 'verb.competition', 'noun.communication', 'noun.attribute', 'noun.act',        │
│           │                    │ 'noun.time', 'noun.feeling', 'noun.state', 'noun.body', 'verb.creation',       │
│           │                    │ 'noun.Tops', 'verb.emotion', 'noun.person', 'noun.food', 'verb.contact',       │
│           │                    │ 'noun.object'}                                                                 │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 2/120     │ noun.cognition     │ {'noun.communication'}                                                         │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 4/120     │ verb.change        │ {'verb.creation', 'noun.location', 'noun.time'}                                │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 5/120     │ noun.artifact      │ {'verb.creation', 'noun.location', 'noun.cognition', 'noun.communication'}     │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 1/120     │ noun.state         │ {'verb.change'}                                                                │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 4/120     │ noun.event         │ {'noun.location', 'verb.possession', 'noun.communication'}                     │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 1/120     │ noun.body          │ {'noun.event'}                                                                 │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 9/120     │ noun.Tops          │ {'noun.time', 'verb.social', 'verb.change', 'noun.location',                   │
│           │                    │ 'noun.communication', 'noun.event', 'noun.artifact'}                           │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 2/120     │ verb.creation      │ {'noun.location'}    

                                                TAKE right to left                                                 
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Frequenza ┃ Sinistra           ┃ Destra                                                                         ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2/120     │ verb.body          │ {'noun.person', 'noun.communication'}                                          │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 24/120    │ noun.communication │ {'noun.phenomenon', 'noun.person', 'noun.cognition', 'noun.group',             │
│           │                    │ 'noun.artifact', 'noun.communication', 'noun.event', 'noun.Tops'}              │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 32/120    │ noun.location      │ {'noun.phenomenon', 'verb.change', 'noun.group', 'noun.object',                │
│           │                    │ 'noun.communication', 'noun.Tops', 'verb.creation', 'noun.event',              │
│           │                    │ 'noun.attribute', 'noun.artifact', 'noun.act'}                                 │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 2/120     │ noun.cognition     │ {'noun.artifact', 'noun.communication'}                                        │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 6/120     │ noun.artifact      │ {'noun.Tops', 'noun.person', 'noun.communication'}                             │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 4/120     │ verb.change        │ {'noun.state', 'noun.Tops', 'noun.communication'}                              │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 1/120     │ verb.possession    │ {'noun.event'}                                                                 │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 1/120     │ verb.consumption   │ {'noun.communication'}                                                         │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 5/120     │ noun.attribute     │ {'noun.location', 'noun.communication', 'verb.contact', 'noun.group'}          │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 1/120     │ noun.food          │ {'noun.communication'}                                                         │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 4/120     │ noun.event         │ {'noun.Tops', 'noun.communication', 'noun.body', 'noun.group'}                 │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 2/120     │ noun.state         │ {'noun.food', 'noun.communication'}                                            │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 6/120     │ verb.creation      │ {'noun.artifact', 'verb.change', 'noun.communication'}                         │
├───────────┼────────────────────┼────────────────────────────────────────────────────────────────────────────────┤
│ 1/120     │ verb.motion        │ {'noun.communication'}                                                         │
├───────────┼────────────────────┼──────────────────────